In [1]:
import math
import random
import time

import numpy as np
import cv2 as cv

import sklearn.cluster

%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
def indexAxes(axes, i, ncols, nrows):
    if( (ncols == 1) and nrows == 1):
        return axes
    elif(ncols == 1 or nrows == 1):
        return axes[i]
    else:
        I = i // ncols
        J = i % ncols
        return axes[I][J]
    
def showImages(images, title=None, ncols=1, nrows=1):
    fig, ax = plt.subplots(ncols=ncols, nrows=nrows)
    for i,img in enumerate(images):
        if(len(img.shape) == 2):
            h,w = img.shape
            c = 1
        else:
            h,w,c = img.shape
        
        I = i // ncols
        J = i % ncols
        
        if(c == 3):
            imgC = cv.cvtColor(img, cv.COLOR_RGB2BGR)
            indexAxes(ax, i, ncols, nrows).imshow(imgC)
        else:
            indexAxes(ax, i, ncols, nrows).imshow(img, cmap=plt.cm.gray)
            
        if(title is not None and i < len(title)):
            indexAxes(ax, i, ncols, nrows).set_title(title[i])
    
    return fig, ax

In [3]:
def grayToRGB(src):
    return np.stack([src.copy(), src.copy(), src.copy()], -1)

In [4]:
imgRaw = cv.imread("bulleye07.png")
imgG = cv.cvtColor(imgRaw, cv.COLOR_RGB2GRAY)
_, imgBin = cv.threshold(imgG, 60, 255, cv.THRESH_OTSU)

In [5]:
fig1, ax1 = showImages([imgRaw, imgG, imgBin], title=["Raw", "Gray", "Binary"], ncols=3)

<IPython.core.display.Javascript object>

In [6]:
sharpenFilter = np.array([[0, -3, 0],
                          [-3, 13, -3],
                          [0, -3, 0]])
imgSharp = cv.filter2D(imgG, -1, sharpenFilter)

In [7]:
_, imgSharpBin = cv.threshold(imgSharp, 60, 255, cv.THRESH_OTSU)

In [8]:
fig2, ax2 = showImages([imgG, imgBin, imgSharp, imgSharpBin], title=["Gray", "Binary", "Sharp", "Sharp Bin"], ncols=4)

<IPython.core.display.Javascript object>

In [9]:
print(f"Gray Contrast:\t\t{imgG.std():.05f}")
print(f"Bin Contrast:\t\t{imgBin.std():.05f}")
print(f"Sharp Contrast:\t\t{imgSharp.std():.05f}")
print(f"Sharp Bin Contrast:\t{imgSharpBin.std():.05f}")

Gray Contrast:		19.79573
Bin Contrast:		112.95257
Sharp Contrast:		64.49207
Sharp Bin Contrast:	91.57413


In [10]:
imgCanny = cv.Canny(imgSharpBin, 1, 25, None ,3)

In [11]:
fig3, ax3 = showImages([imgCanny], ["Canny"])

<IPython.core.display.Javascript object>

In [12]:
lines = cv.HoughLinesP(imgCanny, 1, math.pi/180, 7)

In [13]:
lines.shape

(108, 1, 4)

In [14]:
def drawLines(src, lines, color=[255,0,0]):
    img = src
    for i in range(len(lines)):
        line = lines[i][0]
        img = cv.line(img, (line[0], line[1]), (line[2], line[3]), color=color)
    return img

In [15]:
imgSharpLines = drawLines(grayToRGB(imgCanny), lines)

In [16]:
fig4, ax4 = showImages([imgSharpBin, imgSharpLines], ["Sharp Binary", "Sharp Lines"], ncols=2)

<IPython.core.display.Javascript object>

In [17]:
contours, hierachy = cv.findContours(imgSharpBin, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

In [18]:
len(contours)

7

In [19]:
cnts = sorted(contours, key = cv.contourArea, reverse = True)[:5]
corners = []
for c in cnts:
    # approximate the contour
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.02 * peri, True)
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        corners = approx
        break
corners

array([[[20,  0]],

       [[36,  1]],

       [[37,  3]],

       [[37,  0]]], dtype=int32)

In [20]:
def drawContourColors(src, contours, colorFunc):
    img = src

    for i in range(len(contours)):
        img = cv.drawContours(img, contours, i, colorFunc(i))
        
    return img

In [21]:
def colorF1(index):
    r = random.randint(0,2 ** 24)
    return [(r & 0xFF0000) >> 16, (r & 0x00FF00) >> 8, (r & 0x0000FF)]

In [22]:
#imgSharpCont = cv.drawContours(grayToRGB(np.zeros_like(imgSharpBin)), contours, 0, [255, 0, 0])
imgSharpCont = drawContourColors(grayToRGB(np.zeros_like(imgSharpBin)), contours, colorF1)

In [23]:
fig5, ax5 = showImages([imgSharpBin, imgSharpCont], ["Sharp Binary", "Sharp Contours"], ncols=2)

<IPython.core.display.Javascript object>

In [24]:
# for c in contours:
#     area = cv.contourArea(c)
    
#     x,y,w,h = cv.boundingRect(c)
#     aspectRatio = float(w)/h
#     boundingRectArea = w*h
    
#     convexHull = cv.convexHull(c)
#     convexHullArea = cv.contourArea(convexHull)
#     solidity = area / convexHullArea
    
#     fillness = area / boundingRectArea
    
#     print(f"Area: {area} Aspect Ratio: {aspectRatio} Solidity: {convexHullArea} Fillness: {fillness}")

In [25]:
# def filterContours(contours, fillnessLimit = (0.75, 1.0)):
    
#     filteredCont = []
#     for c in contours:
#         area = cv.contourArea(c)

#         x,y,w,h = cv.boundingRect(c)
        
#         boundingRectArea = w*h
#         fillness = area / boundingRectArea
        
#         aspectRatio = float(w)/h
        
#         if(fillnessLimit[0] < fillness and fillness < fillnessLimit[1]):
#             filteredCont.append(c)
        
#     return filteredCont

In [26]:
# filtConts = filterContours(contours, fillnessLimit=(0.75, 1.0))

In [27]:
# len(filtConts)

In [28]:
hierachy[0]

array([[ 1, -1, -1, -1],
       [ 2,  0, -1, -1],
       [ 3,  1, -1, -1],
       [ 4,  2, -1, -1],
       [ 5,  3, -1, -1],
       [ 6,  4, -1, -1],
       [-1,  5, -1, -1]], dtype=int32)

In [29]:
def getSubtreeCount(hierachy, node): # node [next, prev, child, parent, index]
    count = 0
    childInd = node[2]
    if(childInd == -1):
        return 1
    child = hierachy[node[2]]
    while(True):
        count += getSubtreeCount(hierachy, hierachy[child[4]]) + 1
        if(child[0] == -1):
            break
        child = hierachy[child[0]]
    return count

In [30]:
def hierachySearch(hierachy):
    index = np.arange(len(hierachy[0]))
    h = np.concatenate([hierachy[0], np.expand_dims(index, -1)], 1)
    
    noParents = h[:,3] == -1
    
    noParentsIndex = index[noParents]
    
    counts = []
    
    for i in noParentsIndex:
        counts.append((i,getSubtreeCount(h, h[i])))
    print(counts)
    return max(counts, key=lambda x: x[1])

In [31]:
maxInd, maxCount = hierachySearch(hierachy)
print(maxInd, maxCount)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]
0 1


In [32]:
boundingCont = contours[maxInd]

In [33]:
boundingCont

array([[[37, 38]],

       [[37, 52]]], dtype=int32)

In [34]:
boundingCont.shape

(2, 1, 2)

In [35]:
def drawPixels(src, points, color=[255,0,0]):
    img = src
    img[points[:,1], points[:,0]] = color
    return img

In [36]:
imgBoundPoints = drawPixels(imgRaw.copy(), boundingCont[:,0])

In [37]:
boundingCH = cv.convexHull(boundingCont)

In [38]:
boundingCH

array([[[37, 52]],

       [[37, 38]]], dtype=int32)

In [39]:
imgBoundCH = drawPixels(imgRaw.copy(), boundingCH[:,0])

In [40]:
fig6, ax6 = showImages([imgRaw, imgBoundPoints, imgBoundCH], ["Raw Img", "Bounding Points", "Convex Hull"], ncols=3)

<IPython.core.display.Javascript object>

In [41]:
approxErr1 = 0.01*cv.arcLength(boundingCH, True)
approxPoly1 = cv.approxPolyDP(boundingCH, approxErr1, True)

In [42]:
approxPoly1

array([[[37, 52]],

       [[37, 38]]], dtype=int32)

In [43]:
# cnts = sorted(contours, key = cv2.contourArea, reverse = True)[:5]
# # loop over the contours
# for c in cnts:
#     # approximate the contour
#     peri = cv2.arcLength(c, True)
#     approx = cv2.approxPolyDP(c, 0.01 * peri, True)
#     # if our approximated contour has four points, then we
#     # can assume that we have found our screen
#     if len(approx) == 4:
#         screenCnt = approx
#         break
# screenCnt

In [44]:
imgBoundApprox1 = drawPixels(imgRaw.copy(), approxPoly1[:,0])

In [45]:
approxPoly1[:,0]

array([[37, 52],
       [37, 38]], dtype=int32)

In [46]:
fig7, ax7 = showImages([imgRaw, imgBoundPoints, imgBoundApprox1], ["Raw Img", "Bounding Points", "Bounding Approximation 1"], ncols=3)

<IPython.core.display.Javascript object>

In [47]:
boundCHPoints = boundingCH[:,0]

In [48]:
boundCHPoints

array([[37, 52],
       [37, 38]], dtype=int32)

In [49]:
boundCHDirs = boundCHPoints - np.roll(boundCHPoints, -1, axis=0)
boundCHMids = (boundCHPoints + np.roll(boundCHPoints, -1, axis=0)) / 2

In [50]:
boundCHDirs

array([[  0,  14],
       [  0, -14]], dtype=int32)

In [51]:
boundCHMids

array([[37., 45.],
       [37., 45.]])

In [52]:
boundCHDirsNorm = np.divide(boundCHDirs, np.expand_dims(np.linalg.norm(boundCHDirs, axis=1), 1))

In [53]:
boundCHDirsNorm

array([[ 0.,  1.],
       [ 0., -1.]])

In [54]:
boundCHAngles = np.arctan2(boundCHDirs[:,1], boundCHDirs[:,0])

In [55]:
boundCHAngles

array([ 1.57079633, -1.57079633])

In [56]:
boundCHMag = np.linalg.norm(boundCHDirs, axis=1)

In [57]:
boundCHMag

array([14., 14.])

In [58]:
boundCluster1 = sklearn.cluster.KMeans(n_clusters=4)

In [59]:
boundCluster1.fit(boundCHDirsNorm)

ValueError: n_samples=2 should be >= n_clusters=4.

In [ ]:
boundCluster1.labels_

In [ ]:
boundRefinedAngle = []
boundRefinedMid = []
for i in range(4):
    groupMask = boundCluster1.labels_ == i
    
    groupMag = boundCHMag[groupMask]
    weighting = groupMag / np.sum(groupMag)
    
    groupAngle = boundCHAngles[groupMask]
    boundRefinedAngle.append(np.dot(groupAngle, weighting))
    
    groupMid = boundCHMids[groupMask]
    boundRefinedMid.append(np.sum(np.multiply(groupMid, np.expand_dims(weighting,-1)), 0))

In [ ]:
boundRefinedAngle

In [ ]:
boundRefinedMid

In [ ]:
def rotationMatrix(theta):
    cT = math.cos(theta)
    sT = math.sin(theta)
    rot = np.array([[cT, -sT], [sT, cT]])
    return rot

In [ ]:
def roundClipToEdge(points, maxDim):
    # maxDim = [maxX, maxY]
    return np.clip(np.round(points).astype(np.int32), [0,0], maxDim)

In [ ]:
def drawPointAngle(src, points, angles, length=5, color=[255, 0, 0]):
    img = src
    for p,a in zip(points, angles):
        L = 5//2
        line = np.matmul(rotationMatrix(a), np.stack([np.arange(1,L+1), np.zeros(L)], -1).T).T
        l1 = p + line
        l2 = p - line

        newP = roundClipToEdge(np.concatenate([[p], l1, l2], 0), [src.shape[1], src.shape[0]])
        img = drawPixels(img, newP, color=color)
    
    return img

In [ ]:
imgBoundRefined = drawPointAngle(imgRaw.copy(), boundRefinedMid, boundRefinedAngle)

In [ ]:
fig7, ax7 = showImages([imgBoundRefined], ["Refined"])

In [ ]:
def sortPoints(points, angles):
    # Sort into [top, left, right, bottom]
    
    p = np.array(points)
    print(p)
    top = np.argmin(p[:,1])
    left = np.argmin(p[:,0])
    right = np.argmax(p[:,0])
    bot = np.argmax(p[:,1])
    
    index = [top, left, right, bot]
    
    pointsSort = []
    anglesSort = []
    
    for ind in index:
        pointsSort.append(points[ind])
        anglesSort.append(angles[ind])
    
    return pointsSort, anglesSort

In [ ]:
edgeMidSort, edgeAngleSort = sortPoints(boundRefinedMid, boundRefinedAngle)

In [ ]:
def intersectionLL(p1, p2, a1, a2, limit=0.1, epsilon=1e-3):
    #p1,p2 is column vector [[x],[y]]
    
    # epsilon is for checking if error between intersection from 2 ways of calculation is acceptable or not
    # if not None is returned
    
    # limit is difference in angle in radians. if angle difference is less than limit, None is returned
    if(abs(a1-a2) < limit):
        return None
    
    v1 = np.array([[math.cos(a1)], [math.sin(a1)]])
    v2 = np.array([[math.cos(a2)], [math.sin(a2)]])
    
    A = np.concatenate([v1,-v2], axis=1)
    b = p2 - p1
    x = np.matmul(np.linalg.inv(A), b)
    
    #Check
    k1 = x[0,0]
    k2 = x[1,0]
    
    int1 = k1*v1 + p1
    int2 = k2*v2 + p2
    
    if(np.linalg.norm(int1-int2) > epsilon):
        return None

    return (int1 + int2) / 2

In [ ]:
def getKeypoints(points, angles):
    # accept sorted points and angles
    
    # topLeft, topRight, botLeft, botRight
    indexPair = [(0,1), (0,2), (3,1), (3,2)]
    
    keypoints = []
    for i1,i2 in indexPair:
        keypoints.append(intersectionLL(points[i1].reshape(-1,1), points[i2].reshape(-1,1), angles[i1], angles[i2]))
    
    return keypoints

In [ ]:
kp1 = getKeypoints(edgeMidSort, edgeAngleSort)

In [ ]:
kp1

In [ ]:
fig8, ax8 = showImages([imgRaw, imgBoundRefined, imgRaw], ["Raw", "Refined", "Kps"], ncols=3)
npKp1 = np.array(kp1)
ax8[2].scatter(npKp1[:,0], npKp1[:,1], s=3, c='r')

In [ ]:
import pickle
import cv2

calib_result_pickle = pickle.load(open("new_camera_calib_pickle.p", "rb" ))
mtx = calib_result_pickle["mtx"]
optimal_camera_matrix = calib_result_pickle["optimal_camera_matrix"]
dist = calib_result_pickle["dist"]
roi = calib_result_pickle["roi"]


In [ ]:
def rotationMatrixToEulerAngles(R) :
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])

    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z])

In [ ]:
src_pts = np.array([[13,14], [84,14],[84,84], [14,84]], dtype=float)
src_pts

In [ ]:
imgCrop = cv.imread("symbols/00bulleye.png")

In [ ]:
def find_approx(contours):
# loop over the contours
    cnts = sorted(contours, key = cv2.contourArea, reverse = True)[:5]
    for c in cnts:
        # approximate the contour
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.01 * peri, True)
        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            corners = approx
            break
    return corners
def order_points(pts):

    dst_pts = []
    for i in pts:
        x,y = i.ravel()
        dst_pts.append([x,y])
    dst_pts = np.array(dst_pts)

    # Initialize the order in a clockwise dir (top_left, top_right, btm_right, btm_left)
    rect = np.zeros((4, 2), dtype=float)

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = dst_pts.sum(axis = 1)
    rect[0] = dst_pts[np.argmin(s)]
    rect[2] = dst_pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(dst_pts, axis = 1)
    rect[1] = dst_pts[np.argmin(diff)]
    rect[3] = dst_pts[np.argmax(diff)]

    return rect 

In [ ]:
cnts = sorted(contours, key = cv2.contourArea, reverse = True)[:5]
for c in cnts:
    # approximate the contour
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.01 * peri, True)
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        corners = approx
        break
corners

In [ ]:
sorted_dst_pts = order_points(corners)
sorted_dst_pts

In [ ]:
import cv2
src_key = cv2.KeyPoint_convert(src_pts)
#objectCoord = cv2.KeyPoint_convert(dst_match_01)
dst_key = cv2.KeyPoint_convert(sorted_dst_pts)

matches = []

for i in range(len(src_key)):
    match = cv2.DMatch()
    match.queryIdx = i
    match.trainIdx = i
    matches.append(match)

kpMatchImg = cv2.drawMatches(imgCrop, src_key, imgRaw, dst_key,matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("KF", kpMatchImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
H, status = cv2.findHomography(src_pts, sorted_dst_pts)
# H, status = cv2.findHomography(src_match, dst_match)
H

In [ ]:
num, Rs, Ts, Ns  = cv2.decomposeHomographyMat(H, optimal_camera_matrix)
im_out = cv2.warpPerspective(imgCrop, H, (100, 100))
cv2.imshow("LOL", im_out)

for rotVec, tvec in zip(Rs, Ts):
    eulerAngle = rotationMatrixToEulerAngles(rotVec)
    print(f"\nRotation Vector: \n{eulerAngle * 180/math.pi}\n \nTranslation\n {tvec}")

cv2.waitKey(0)

In [65]:
data = "00,1,1,12,0,0"

split_data = data.split(",")
for index, value in enumerate(split_data):
    if value == "00":
        x_correct = split_data[index+1]
        y_correct = split_data[index+2]
        print(x_correct)
        print(y_correct)

1
1


In [74]:
image_id = "21"
x_correct = "0"
y_correct = "10"

data = []
sendStr = image_id + ',' + x_correct +','+ y_correct
data.append(sendStr)

In [80]:
image_id = "00"
x_correct = "1"
y_correct = "7"
angle_1 = "31"
sendStr_01 = image_id + ',' + x_correct +','+ y_correct + ',' + angle_1
data.append(sendStr_01)

In [79]:
data

['21,0,10', '00,1,7']

In [78]:
hello = "40" + "0" + "0" + "0"
hello

'40000'

In [81]:
data

['21,0,10', '00,1,7', '00,1,7,31']

In [91]:
image_id = "40Stop"
real_X = 1
real_Y = 7
angle = 13.93123
incre = image_id[:2] + "," + str(real_X) + "," + str(real_Y) + "," + str(int(angle))
data_set = []
data_set.append(incre)

In [92]:
data_set

['40,1,7,13']